## 模型4 resnet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

%matplotlib qt

In [2]:
batch_size = 32
model_accuracies = {
    'resnet': []
}
# 数据增强和预处理
transform4 = transforms.Compose([
    transforms.Resize((224, 224)),  # 图像大小转化
    transforms.ToTensor(),  # 将图像转换为PyTorch的张量格式，并将像素值从0-255缩放到0-1之间
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

In [3]:
# 加载数据集
data_dir = r'D:\深度学习练习\flower_photos\flower_photos'
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform4)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):#残差块定义
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()# 残差路径
        if stride != 1 or in_channels != self.expansion*out_channels:# 如果步长不为1或输入通道数与输出通道数（扩展后）不匹配，则需要改变残差路径的维度
            self.shortcut = nn.Sequential(  # 残差路径的卷积层，用于调整通道数和进行下采样
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=5):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        # 通过添加池化层计算展平后的特征图大小
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 使特征图的尺寸变为 (1, 1)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """
    构建一个包含多个残差块的层。

    参数:
    - block (nn.Module): 残差块的基本构建单元。
    - out_channels (int): 残差块输出特征图的通道数。
    - num_blocks (int): 层中残差块的数量。
    - stride (int): 第一个残差块的卷积步长。

    返回:
    - nn.Sequential: 包含多个残差块的层。
    """
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        # 使用自适应池化将特征图调整为 (1, 1)
        out = self.avgpool(out)
        
        # 展平特征图
        out = torch.flatten(out, 1)  # 展平为 (batch_size, num_features)
        
        out = self.linear(out)
        return out

def ResNet18(num_classes=5):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)




In [5]:
# 参数设置
batch_size = 32
learning_rate = 0.0001
epochs = 40

# 检查GPU是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 初始化模型、损失函数和优化器
# 创建模型实例
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
accuracies = []

# 训练模型
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到GPU
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    # 将准确率存储到 model_accuracies 字典中
    model_accuracies['resnet'].append(accuracy)
    print(f'Epoch [{epoch+1}/{epochs}], Validation Accuracy: {accuracy:.2f}%')

# 计算验证集的准确率
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Using device: cuda
Epoch 1, Train Loss: 1.1991146537272825
Epoch [1/40], Validation Accuracy: 57.63%
Epoch 2, Train Loss: 0.967334267237912
Epoch [2/40], Validation Accuracy: 64.31%
Epoch 3, Train Loss: 0.8557030584501184
Epoch [3/40], Validation Accuracy: 67.17%
Epoch 4, Train Loss: 0.8060407894461051
Epoch [4/40], Validation Accuracy: 65.94%
Epoch 5, Train Loss: 0.7555098727993343
Epoch [5/40], Validation Accuracy: 63.62%
Epoch 6, Train Loss: 0.7258848062028056
Epoch [6/40], Validation Accuracy: 65.26%
Epoch 7, Train Loss: 0.6905408261910729
Epoch [7/40], Validation Accuracy: 64.03%
Epoch 8, Train Loss: 0.6345418299669805
Epoch [8/40], Validation Accuracy: 60.08%
Epoch 9, Train Loss: 0.612623516632163
Epoch [9/40], Validation Accuracy: 73.43%
Epoch 10, Train Loss: 0.5934041555485
Epoch [10/40], Validation Accuracy: 69.75%
Epoch 11, Train Loss: 0.5532868983952896
Epoch [11/40], Validation Accuracy: 65.94%
Epoch 12, Train Loss: 0.5435774122243342
Epoch [12/40], Validation Accuracy: 72.

In [6]:

# 保存模型权重
torch.save(model.state_dict(), 'resnet_weights.pth')
# 保存模型结构和权重
torch.save(model, 'resnet_model.pth')
accuracies_list = model_accuracies['resnet']

# 创建一个包含epoch编号和对应准确率的字典
data = {'Epoch': list(range(1, len(accuracies_list) + 1)),
        'Accuracy': accuracies_list}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 现在你可以将DataFrame保存为Excel文件
df.to_excel('resnet_accuracies.xlsx', index=False)
# 清理内存
del model
del inputs
del labels
torch.cuda.empty_cache()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 实例化模型
model = ResNet18(num_classes=5).to(device)

# 加载模型权重
model.load_state_dict(torch.load('resnet_weights.pth'))

# 设置为评估模式
model.eval()

Using device: cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [8]:
import torch
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# 确保模型处于评估模式
model.eval()

# 初始化用于存储预测和真实标签的列表
all_preds = []
all_labels = []

# 不计算梯度进行预测
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 将数据移动到GPU，如果可用
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测概率最高的类别
        _, predicted = torch.max(outputs.data, 1)

        # 将预测结果和真实标签添加到列表中
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 计算准确率、精确率、召回率和F1分数
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

# 将结果保存到CSV文件
results = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# 写入CSV文件
with open('ResNet评估.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Value'])
    for metric, value in results.items():
        writer.writerow([metric, value])

print("Results have been saved to ResNet评估.csv")

Evaluating: 100%|██████████| 23/23 [00:04<00:00,  5.64it/s]

Results have been saved to ResNet评估.csv
